# Introduction
## (Author: Nathan Brunetti)
This notebook will walk you through using the ALMA archive query enhanced for identifying spectral lines potentially observed. It works through an example looking for observations from a circular region of radius 6&deg; centered on the Virgo Cluster in a redshift range of $0.005 \leq z \leq 0.1$ which may contain CO (J=2-1) and CN (N=1-0). The `archiveSearch.runQueriesWithLines` convenience function does most of the work so the remainder of the notebook explores the results tables and checks for projects and sources that may have had both lines observed.

The underlying approach wrapped up in the `archiveSearch.runQueriesWithLines` function is to 
1. calculate the frequency range that contains our lines, in the given redshift range
1. query the ALMA archive region using that frequency range
1. match the source names in the ALMA archive against names in the NED database
1. use the redshift from NED to calculate the observed frequency of our lines for each source
1. check if the ALMA spectral window coverage contains the observed line frequencies

### To do for this notebook
* Display project codes at the end for sources which have both lines observed but are necessarily done in different projects. It is hard to see which projects are listed in the big full table so make it more human-accessible.
* Add exploration of tables where no there are no NED matches and where NED matches but does not have redshift.

# Import modules we use

In the short-term it is necessary to install the "bleeding edge" version of `astroquery` to avoid a bug in the release date column. To do so, run the following commands in a terminal in a directory you do not mind downloading ~100 MB of code
```
git clone https://github.com/astropy/astroquery.git
cd astroquery
python setup.py build
python setup.py install
```

Imports of external modules.

In [1]:
from astropy.table import vstack
from astroquery.ned import Ned
import numpy as np
import os
import sys

Import the enhanced ALMA query class, called `ALMAxmatch.archiveSearch`.

In [2]:
sys.path.insert(0, os.path.abspath('..'))
from ALMAxmatch import archiveSearch

# Run the archive query

Since we have a particular target that we want to query on, we initialize the `archiveSearch` with that target. In this case, we specify the target as a region defined by a tuple containing the coordinates of the region center and a search radius.

In [3]:
virgoLineQuery = archiveSearch(targets=[['12h26m32.1s 12d43m24s', '6deg']])

Now we have an `archiveSearch` instance called `virgoLineQuery` that contains our desired search target. Running the actual archive query to include the search for lines is almost as easy as one without a line search. All of the same arguments that `astroquery`'s ALMA query tool accepts can be used here (e.g. science, band, polarization, etc.) except the frequency argument. This is because the `archiveSearch` class uses the frequency parameter to limit the search to where the lines may appear.

We have to specify the rest frequencies of the spectral lines we are interested in to do the line search (names are optional, output defaults to *line 0*, *line 1*, ...). A redshift range is optional and we choose to use one here. In this case, we also choose to limit the archive search to only science targets (excluding calibrators). This is fine because we want local galaxies!

In [4]:
virgoLineQuery.runQueriesWithLines([230.538, 113.123337],
                                   redshiftRange=(0.005, 0.1),
                                   lineNames=['CO (J=2-1)', 'CN (N=1-0)'],
                                   science=True)

And that's it! Stored within `virgoLineQuery` are the ALMA observations in the specified region, with spectral coverage set by the rest fruquencies of the lines and redshift range, as well as whether CN and CO fall in the observed frequency coverage. There are also tables that contain the ALMA observations that fell in the frequency range but did not match any objects in NED by name and for objects that were found in NED but did not have a redshift.

So all of the ALMA observations in that region within that frequency range are stored in `virgoLineQuery`. These tables can be accessed as 
* `virgoLineQuery.queryResults`
* `virgoLineQuery.queryResultsNoNED`
* `virgoLineQuery.queryResultsNoNEDz`.

# Find observations with both CN and CO
The rest of this notebook will deal with manipulating the archive query results to find information that may be useful. The first goal will be to find what observations (if any) have both CN and CO spectral coverage for the object's redshift.

We will just set a couple convenience variables here to make accessing the table easier, since there was only one target in our query.

In [5]:
print(virgoLineQuery.targets)

{'coord=(186.63374999999996 deg 12.723333333333333 deg) radius=6.0deg': [<SkyCoord (ICRS): (ra, dec) in deg
    (186.63375, 12.72333333)>, <Angle 6. deg>]}


In [6]:
target = 'coord=(186.63374999999996 deg 12.723333333333333 deg) radius=6.0deg'
searchTable = virgoLineQuery.queryResults[target]

Let us take a look at the whole results table for observations that had at least one of the lines covered. Two columns all the way on the right side of the table with our given line names "CO (J=2-1)" and "CN (N=1-0)" specify if each line was covered in the spectral setup. Also here are all of the standard ALMA archive data columns, with some extras added by the `archiveSearch` class to make the table a bit more machine-capable (e.g. "Frequency ranges" giving all of the SPW frequency ranges as a list of lists rather than the default mile-long string).

In [7]:
searchTable.show_in_notebook()

idx,Project code,ALMA source name,ALMA RA,ALMA Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges,ALMA sanitized source name,NED source name,NED RA,NED Dec,NED Redshift,CO (J=2-1),CN (N=1-0)
,,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcsec,arcsec,,,,,,GHz,,,degrees,degrees,,,
0,2011.0.00004.SV,M100,185.72641646606877,15.814268122637317,271.14507499320354,76.89071210978545,[3],11.255079108695867,976.5877688782356,7m,mosaic,121.871,2016-06-24,"[99.96..101.95GHz,976.59kHz,53.9mJy/beam@10km/s,2.2mJy/beam@native, XX YY] U [101.77..103.76GHz,976.59kHz,54.1mJy/beam@10km/s,2.2mJy/beam@native, XX YY] U [111.81..113.81GHz,976.59kHz,67.4mJy/beam@10km/s,2.9mJy/beam@native, XX YY] U [113.69..115.68GHz,976.59kHz,86.4mJy/beam@10km/s,3.8mJy/beam@native, XX YY]",2530.936294484596,XX YY,2013-04-14T03:41:35,"observatory, ALMA",M100 ACA,,54.14744615801012,1.3042760579068018,5.186,uid://A002/X5d9e5c/X3eb,uid://A002/X5d9e5c/X3ec,uid://A002/X60b415/X44,Science verification observation of M100,SV,TARGET,236.34320344434877,44.221507264719605,Y,6,"Spiral galaxies, Giant Molecular Clouds (GMC) properties",Local Universe,2011.0.00004.SV,"[array([ 99.96, 101.95]), array([101.77, 103.76]), array([111.81, 113.81]), array([113.69, 115.68])]",M100,MESSIER 100,185.72846,15.82182,0.00524,False,True
1,2011.0.00004.SV,M100,185.72730240368028,15.815548848881216,271.14559959493437,76.89224624437853,[3],2.6096705795833137,976.5660127571381,12m7m,mosaic,251.137,2016-06-24,"[99.98..101.85GHz,976.57kHz,13.5mJy/beam@10km/s,570.2uJy/beam@native, XX YY] U [101.79..103.66GHz,976.57kHz,13.4mJy/beam@10km/s,573.9uJy/beam@native, XX YY] U [111.85..113.73GHz,976.57kHz,16.7mJy/beam@10km/s,747.1uJy/beam@native, XX YY] U [113.73..115.60GHz,976.57kHz,22mJy/beam@10km/s,991.7uJy/beam@native, XX YY]",2532.567196365089,XX YY,2011-09-10T18:11:35,"observatory, ALMA",NGC4321 (M100) in az,,13.425764887555946,0.3348327007910755,0.5855137,,uid://A002/X259150/X130,uid://A002/X2a5c2f/X220,Science verification observation of M100,SV,TARGET WVR,260.818629595427,21.807805306235075,Y,6,"Spiral galaxies, Giant Molecular Clouds (GMC) properties",Local Universe,2011.0.00004.SV,"[array([ 99.98, 101.85]), array([101.79, 103.66]), array([111.85, 113.73]), array([113.73, 115.6 ])]",M100,MESSIER 100,185.72846,15.82182,0.00524,False,True
2,2011.0.00004.SV,M100,185.72641646606877,15.814268122637323,271.14507499320354,76.89071210978544,[3],10.417732879675647,976.545993209982,7m,mosaic,246.641,2016-06-24,"[111.80..113.79GHz,976.55kHz,58.3mJy/beam@10km/s,2.5mJy/beam@native, XX YY] U [113.67..115.66GHz,976.55kHz,74.5mJy/beam@10km/s,3.3mJy/beam@native, XX YY]",2531.113571079122,XX YY,2013-03-18T05:22:12,"observatory, ALMA",M100 ACA,,58.30022274858676,2.001676562382841,5.186,uid://A002/X5d9e5c/Xb7,uid://A002/X5d9e5c/Xb8,uid://A002/X5e9ff1/X3f3,Science verification observation of M100,SV,TARGET,233.30638244786167,44.57019990611818,Y,6,"Spiral galaxies, Giant Molecular Clouds (GMC) properties",Local Universe,2011.0.00004.SV,"[array([111.8 , 113.79]), array([113.67, 115.66])]",M100,MESSIER 100,185.72846,15.82182,0.00524,False,True
3,2011.0.00004.SV,M100,185.72569357167404,15.814568156319671,271.14173961725174,76.89063547966255,[3],2.26359351555494,976.6086804828678,12m,mosaic,72.675,2016-06-24,"[99.98..101.86GHz,976.61kHz,17.9mJy/beam@10km/s,756.5uJy/beam@native, XX YY] U [101.79..103.67GHz,976.61kHz,17.8mJy/beam@10km/s,761.9uJy/beam@native, XX YY] U [111.86..113.73GHz,976.61kHz,22.2mJy/beam@10km/s,992.3uJy/beam@native, XX YY] U [113.73..115.61GHz,976.61kHz,29.1mJy/beam@10km/s,

We can do a quick check of the number of observations that have CN, CO, or both observed. ```np.sum``` treats ```True``` as 1 and ```False``` as 0 so the sum gets us what we want. There are no observations with both lines which sounds disappointing until you remember that each observation is only in a single band and these lines are too far apart to be in the same band for the redshift range we specified (not until $z\approx14$ are they $\leq$ 8 GHz apart).

In [8]:
print('No. observations with CO(2-1): ', np.sum(searchTable['CO (J=2-1)']))
print('No. observations with CN(1-0): ', np.sum(searchTable['CN (N=1-0)']))
print('No. observations with both: ', np.sum((searchTable['CO (J=2-1)']) & (searchTable['CN (N=1-0)'])))

No. observations with CO(2-1):  99
No. observations with CN(1-0):  19
No. observations with both:  0


So we need to do the check a little differently to find sources that were observed in both lines. How about we check based on project?

In [9]:
CO21observed = searchTable['CO (J=2-1)']
CN10observed = searchTable['CN (N=1-0)']

COprojects = np.unique(searchTable['Project code'][CO21observed])
CNprojects = np.unique(searchTable['Project code'][CN10observed])

CO projects.

In [10]:
print(COprojects)

 Project code 
--------------
2012.1.00139.S
2013.1.00073.S
2013.1.00828.S
2015.1.00466.S
2015.1.00598.S
2015.1.00956.S
2015.1.01056.S
2015.1.01341.S
2016.1.00912.S
2016.1.01154.V
2016.2.00046.S
2016.2.00053.S
2017.1.00297.S
2017.1.00301.S
2017.1.00608.S
2017.1.00766.S
2017.1.00841.V
2017.1.00886.L
2017.1.00964.S
2018.1.00484.S


CN projects.

In [11]:
print(CNprojects)

  Project code 
---------------
2011.0.00004.SV
 2012.1.00661.S
 2012.1.00759.S
 2015.1.00030.S
 2015.1.00820.S
 2015.1.00978.S
 2015.1.00989.S
 2015.1.01161.S
 2016.1.00021.S
 2016.1.00972.S
 2018.1.01599.S


Projects with both lines. Still none! I am not out of tricks yet though.

In [12]:
print(np.intersect1d(np.array(CNprojects), np.array(COprojects)))

[]


This time, let us find the ALMA source names that had both lines observed even if the observations come from different projects. Here we use another of the `archiveSearch` class added column names called "ALMA sanitized source name" which just took the normal "Source name" column that comes out of the archive, removed whitespace, removed underscores, and made all letter uppercase. This makes it possible to match projects where the PI named something like "m 87" to another with "M87" (which is something I think the ALMA archive team should do...).

In [13]:
COsources = np.unique(searchTable['ALMA sanitized source name'][CO21observed])
CNsources = np.unique(searchTable['ALMA sanitized source name'][CN10observed])
lineIntersectSources = np.intersect1d(np.array(CNsources), np.array(COsources))

Sources with CO.

In [14]:
print(COsources)

   None   
----------
       M87
   NGC4168
   NGC4207
   NGC4235
   NGC4254
   NGC4293
   NGC4298
   NGC4321
   NGC4374
   NGC4388
       ...
   NGC4540
   NGC4548
   NGC4552
   NGC4569
   NGC4571
   NGC4579
   NGC4596
   NGC4654
   NGC4689
   NGC4694
PG1211+143
Length = 34 rows


Sources with CN.

In [15]:
print(CNsources)

  None 
-------
   M100
    M87
NGC4321
NGC4396
NGC4451
NGC4477
NGC4526
NGC4596
  VV219


Sources with both lines, and this time we have victory.

In [16]:
print(lineIntersectSources)

['M87' 'NGC4321' 'NGC4477' 'NGC4596']


Now we know we have several ALMA source names that have both so let us grab the full archive result rows for those.

In [17]:
COandCNobs = dict()
for sourceName in lineIntersectSources:
    COobs = searchTable[CO21observed]
    COobs = COobs.group_by('ALMA sanitized source name')
    mask = COobs.groups.keys['ALMA sanitized source name'] == [sourceName]
    COobs = COobs.groups[mask]

    CNobs = searchTable[CN10observed]
    CNobs = CNobs.group_by('ALMA sanitized source name')
    mask = CNobs.groups.keys['ALMA sanitized source name'] == [sourceName]
    CNobs = CNobs.groups[mask]

    COandCNobs[sourceName] = vstack([COobs.group_by('Project code'),
                                     CNobs.group_by('Project code')])

M87.

In [18]:
display(COandCNobs[lineIntersectSources[0]])

Project code,ALMA source name,ALMA RA,ALMA Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges,ALMA sanitized source name,NED source name,NED RA,NED Dec,NED Redshift,CO (J=2-1),CN (N=1-0)
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcsec,arcsec,,,,,,GHz,,,degrees,degrees,,,
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object,str23,bytes30,float64,float64,float64,bool,bool
2013.1.00073.S,M87,187.70593075,12.39112331,283.7778185352056,74.49110901444618,[6],0.4493429353061913,976.6406355605037,12m,,876.96,2016-10-27,"[213.00..214.99GHz,31252.50kHz,934.8uJy/beam@10km/s,55.8uJy/beam@native, XX YY] U [215.00..216.99GHz,31252.50kHz,932.1uJy/beam@10km/s,55.9uJy/beam@native, XX YY] U [228.63..230.50GHz,976.64kHz,942.3uJy/beam@10km/s,60.2uJy/beam@native, XX YY] U [230.57..232.56GHz,31252.50kHz,947.4uJy/beam@10km/s,58.9uJy/beam@native, XX YY]",1270.3341995652788,XX YY,2015-06-14T23:44:32,"Tan, Jonathan",M87_a_06_TC,"Beuther, Henrik; Koda, Jin; Blackman, Eric; Walter, Fabian; Kong, Shuo;",0.9320987688472917,0.02881471591241235,0.55976033,uid://A001/X12f/X20c,uid://A001/X12f/X20f,uid://A002/Xa39269/X11e,A Search for Molecular Gas in the Circumnuclear Disk of M87,S,TARGET,28.27877095979075,2.107371622374515,Y,0,"Active Galactic Nuclei (AGN)/Quasars (QSO), Galactic centres/nuclei",Active galaxies,2013.1.00073.S,"[array([213. , 214.99]), array([215. , 216.99]), array([228.63, 230.5 ]), array([230.57, 232.56])]",M87,MESSIER 087,187.70593,12.39112,0.004283,True,False
2013.1.00073.S,M87,187.70593075,12.39112331,283.7778185352056,74.49110901444618,[6],0.202893381974251,976.6060985261313,12m,,1753.92,2016-12-09,"[213.00..214.99GHz,31251.40kHz,862.8uJy/beam@10km/s,51.5uJy/beam@native, XX YY] U [215.00..216.99GHz,31251.40kHz,860.3uJy/beam@10km/s,51.6uJy/beam@native, XX YY] U [228.63..230.50GHz,976.61kHz,874.7uJy/beam@10km/s,55.9uJy/beam@native, XX YY] U [230.57..232.56GHz,31251.40kHz,880.3uJy/beam@10km/s,54.7uJy/beam@native, XX YY]",1270.2446029386717,XX YY,2015-08-16T18:47:01,"Tan, Jonathan",M87_a_06_TE,"Beuther, Henrik; Koda, Jin; Blackman, Eric; Walter, Fabian; Kong, Shuo;",0.8603100972038163,0.02667262878158341,1.768391,uid://A001/X12f/X20c,uid://A001/X12f/X20d,uid://A002/Xa830fc/X43bc,A Search for Molecular Gas in the Circumnuclear Disk of M87,S,TARGET,28.2787724130583,1.0692166493291724,Y,0,"Active Galactic Nuclei (AGN)/Quasars (QSO), Galactic centres/nuclei",Active galaxies,2013.1.00073.S,"[array([213. , 214.99]), array([215. , 216.99]), array([228.63, 230.5 ]), array([230.57, 232.56])]",M87,MESSIER 087,187.70593,12.39112,0.004283,True,False
2016.1.01154.V,M87,187.70593075001403,12.39112331000213,283.77781853525215,74.4911090144523,[6],1.316390675515377,15624.832899089339,12m,,4435.2,2018-10-21,"[212.16..214.04GHz,15624.98kHz,592.9uJy/beam@10km/s,36.5uJy/beam@native, XX XY YY] U [214.16..216.04GHz,15624.98kHz,606.9uJy/beam@10km/s,37.5uJy/beam@native, XX XY YY] U [226.16..228.04GHz,15624.98kHz,584.4uJy/beam@10km/s,37.1uJy/beam@native, XX XY YY] U [228.16..230.04GHz,15624.98kHz,584.7uJy/beam@10km/s,37.3uJy/beam@native, XX XY YY]",20363.14341873059,XX XY YX YY,2017-04-05T03:27:22,"Consortium, Event Horizon Telescope",95d_M87_a_06_TM1,"Muders, Dirk; Asada, Keiichi; Conway, John; Krichbaum, Thomas; Bremer, Michael; Huang, Chih-Wei; Nagai, Hiroshi; Weintroub, Jonathan ; Ca

NGC4321.

In [19]:
display(COandCNobs[lineIntersectSources[1]])

Project code,ALMA source name,ALMA RA,ALMA Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges,ALMA sanitized source name,NED source name,NED RA,NED Dec,NED Redshift,CO (J=2-1),CN (N=1-0)
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcsec,arcsec,,,,,,GHz,,,degrees,degrees,,,
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object,str23,bytes30,float64,float64,float64,bool,bool
2015.1.00956.S,NGC_4321,185.72585845347993,15.83502065988929,271.0983950196347,76.90856907422035,[6],6.106590027576247,488.2992828591299,7m,mosaic,122.948,3000-01-01,"[214.26..216.25GHz,31251.58kHz,75.5mJy/beam@10km/s,4.5mJy/beam@native, XX YY] U [216.80..218.80GHz,3906.45kHz,74.5mJy/beam@10km/s,4.5mJy/beam@native, XX YY] U [228.83..229.83GHz,488.31kHz,74.9mJy/beam@10km/s,6.5mJy/beam@native, XX YY] U [229.99..231.98GHz,31251.58kHz,76.5mJy/beam@10km/s,4.7mJy/beam@native, XX YY]",636.9640593977437,XX YY,2016-07-25T22:57:56,"Leroy, Adam",NGC_4321_a_06_7M,"Bolatto, Alberto; Garcia-Burillo, Santiago; Sliwa, Kazimierz; Groves, Brent; Kruijssen, Diederik; Renaud, Florent; Pety, Jérôme; Schruba, Andreas; Glover, Simon; Sandstrom, Karin; Bigiel, Frank; Chandar, Rupali; Rosolowsky, Erik; Colombo, Dario; Longmore, Steven; Munoz-Mateos, Juan Carlos; Walter, Fabian; Guzman, Andres Ernesto; Klessen, Ralf; Hughes, Annie; Thompson, Todd; Kravtsov, Andrey; Rebolledo, David; Usero, Antonio; Blanc, Guillermo; Meidt, Sharon; Escala, Andres; Schinnerer, Eva; de Blok, W.J.G.",74.50355286695498,2.4528676812746673,1.8,uid://A001/X2fb/X2b8,uid://A001/X2fb/X2bb,uid://A002/Xb5ee5a/X204,How Does Cloud-Scale Physics Drive Galaxy Evolution?,S,TARGET,228.40176491694416,22.71845031107414,Y,3,"Spiral galaxies, Surveys of galaxies",Local Universe,2015.1.00956.S,"[array([214.26, 216.25]), array([216.8, 218.8]), array([228.83, 229.83]), array([229.99, 231.98])]",NGC4321,MESSIER 100,185.72846,15.82182,0.00524,True,False
2015.1.00956.S,NGC_4321,185.72585845342243,15.83487984748789,271.0987022811742,76.9084460571592,[6],1.1349450253087128,488.2401324912066,12m,mosaic,62.282,2017-05-11,"[214.26..216.25GHz,31247.37kHz,7.4mJy/beam@10km/s,441.3uJy/beam@native, XX YY] U [216.87..218.74GHz,3905.92kHz,7.3mJy/beam@10km/s,452.8uJy/beam@native, XX YY] U [228.86..229.80GHz,488.24kHz,7.3mJy/beam@10km/s,661.6uJy/beam@native, XX YY] U [229.99..231.98GHz,31247.37kHz,7.5mJy/beam@10km/s,464.3uJy/beam@native, XX YY]",636.8958999513395,XX YY,2016-02-01T09:18:58,"Leroy, Adam",NGC_4321_a_06_TE,"Bolatto, Alberto; Garcia-Burillo, Santiago; Sliwa, Kazimierz; Groves, Brent; Kruijssen, Diederik; Renaud, Florent; Pety, Jérôme; Schruba, Andreas; Glover, Simon; Sandstrom, Karin; Bigiel, Frank; Chandar, Rupali; Rosolowsky, Erik; Colombo, Dario; Longmore, Steven; Munoz-Mateos, Juan Carlos; Walter, Fabian; Guzman, Andres Ernesto; Klessen, Ralf; Hughes, Annie; Thompson, Todd; Kravtsov, Andrey; Rebolledo, David; Usero, Antonio; Blanc, Guillermo; Meidt, Sharon; Escala, Andres; Schinnerer, Eva; de Blok, W.J.G.",7.2734425969949985,0.24299526626940773,1.9222233,uid://A001/X2fb/X2b8,uid://A001/X2fb/X2b9,uid://A002/Xafb2c8/Xaeb,How Does Cloud-Scale Physics Drive Galaxy Evolution?,S,TARGET,235.8145348810567,5.984586653787221,Y,3,"Spiral galaxies, Surveys of galaxies",Local Universe,2015.1.00956.S,"[array([214.26, 216.25]), array([216.87, 218.74]), array([228.86,

NGC4477

In [20]:
display(COandCNobs[lineIntersectSources[2]])

Project code,ALMA source name,ALMA RA,ALMA Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges,ALMA sanitized source name,NED source name,NED RA,NED Dec,NED Redshift,CO (J=2-1),CN (N=1-0)
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcsec,arcsec,,,,,,GHz,,,degrees,degrees,,,
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object,str23,bytes30,float64,float64,float64,bool,bool
2012.1.00139.S,NGC4477,187.50905,13.636442000000002,281.54417742921,75.61188440438005,[6],0.5539252590210176,31252.173915505424,12m,,60.48,2017-01-29,"[211.04..213.02GHz,31252.45kHz,5.1mJy/beam@10km/s,301.2uJy/beam@native, XX YY] U [213.02..215.01GHz,31252.45kHz,5.1mJy/beam@10km/s,301.9uJy/beam@native, XX YY] U [227.03..229.01GHz,31252.45kHz,5mJy/beam@10km/s,307.1uJy/beam@native, XX YY] U [229.03..231.01GHz,31252.45kHz,5mJy/beam@10km/s,309.5uJy/beam@native, XX YY]",40560.53210012937,XX YY,2014-05-21T03:45:43,"Doi, Akihiro",Multi-source-B6a,"Akiyama, Kazunori; Sawada-Satoh, Satoko; Kawaguchi, Toshihiro; Ozaki, Shinobu; Kohno, Kotaro; Hada, Kazuhiro; Terashima, Yuichi; Nakanishi, Kouichiro;",4.979359872100862,0.15243740190324095,0.6638611,uid://A002/X5a9a13/Xe8,uid://A002/X5a9a13/Xe9,uid://A002/X828760/X398,Nuclear Continuum Spectra of nearby low-luminosity AGNs,S,TARGET WVR,28.50389509800761,3.1126947418091504,Y,0,"Active Galactic Nuclei (AGN)/Quasars (QSO), Galactic centres/nuclei",Active galaxies,2012.1.00139.S,"[array([211.04, 213.02]), array([213.02, 215.01]), array([227.03, 229.01]), array([229.03, 231.01])]",NGC4477,NGC 4477,187.50916,13.6366,0.004463,True,False
2017.1.00766.S,NGC4477,187.50813822842392,13.636532622701255,281.54069457063395,75.61167367315862,[6],4.799625699693015,976.6360737506218,7m,mosaic,6718.447,2020-03-16,"[215.24..217.22GHz,31252.43kHz,11.6mJy/beam@10km/s,698.5uJy/beam@native, XX YY] U [217.04..219.04GHz,1953.28kHz,11.6mJy/beam@10km/s,700.4uJy/beam@native, XX YY] U [229.07..230.07GHz,976.64kHz,11.9mJy/beam@10km/s,1mJy/beam@native, XX YY] U [229.92..231.91GHz,31252.43kHz,11.8mJy/beam@10km/s,734uJy/beam@native, XX YY]",1272.7121983886675,XX YY,2018-06-05T21:55:20,"Chevance, Melanie",NGC4477_a_06_7M,"Schruba, Andreas; Jeffreson, Sarah; Leroy, Adam; Bigiel, Frank; Meidt, Sharon; Kruijssen, Diederik; Schinnerer, Eva; Hygate, Alex; Kreckel, Kathryn; Rosolowsky, Erik;",11.615195341959168,0.3815129394406339,1.8,uid://A001/X1273/X655,uid://A001/X1273/X658,uid://A002/Xce574d/X3cf3,From the main sequence to the red cloud: linking the molecular cloud lifecycle to galaxy evolution,S,TARGET,50.27886210093388,21.116284397610254,Y,0,"Early-type galaxies, Giant Molecular Clouds (GMC) properties",Galaxy evolution,2017.1.00766.S,"[array([215.24, 217.22]), array([217.04, 219.04]), array([229.07, 230.07]), array([229.92, 231.91])]",NGC4477,NGC 4477,187.50916,13.6366,0.004463,True,False
2017.1.00766.S,NGC4477,187.50905000002362,13.636441654913396,281.5441778772964,75.61188390127346,[6],0.5352274458375045,976.6047711536718,12m,mosaic,873.358,NaT,"[215.24..217.22GHz,31251.35kHz,1.2mJy/beam@10km/s,73.9uJy/beam@native, XX YY] U [217.11..218.98GHz,1953.21kHz,1.2mJy/beam@10km/s,76.5uJy/beam@native, XX YY] U [229.10..230.04GHz,976.60kHz,1.2mJy/beam@10km/s,112.9uJy/beam@native, XX YY] U [229.92..231.91GHz,31251.35kHz,1.2mJy/beam@10km/s,77.1uJy/beam@native, XX YY]",1272.7987643852427,XX

NGC4596

In [21]:
display(COandCNobs[lineIntersectSources[3]])

Project code,ALMA source name,ALMA RA,ALMA Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges,ALMA sanitized source name,NED source name,NED RA,NED Dec,NED Redshift,CO (J=2-1),CN (N=1-0)
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcsec,arcsec,,,,,,GHz,,,degrees,degrees,,,
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object,str23,bytes30,float64,float64,float64,bool,bool
2017.1.00766.S,NGC4596,189.98254532162449,10.176611271352046,293.29703361598933,72.8326827683984,[6],5.434293012331888,976.5900162706967,7m,mosaic,5429.221,2020-03-22,"[214.82..216.80GHz,31251.61kHz,12.9mJy/beam@10km/s,773.6uJy/beam@native, XX YY] U [216.63..218.62GHz,1953.23kHz,12.8mJy/beam@10km/s,773.8uJy/beam@native, XX YY] U [228.65..229.65GHz,976.61kHz,12.9mJy/beam@10km/s,1.1mJy/beam@native, XX YY] U [229.51..231.49GHz,31251.61kHz,12.9mJy/beam@10km/s,800.9uJy/beam@native, XX YY]",1274.885493662713,XX YY,2018-07-17T22:51:13,"Chevance, Melanie",NGC4596_a_06_7M,"Schruba, Andreas; Jeffreson, Sarah; Leroy, Adam; Bigiel, Frank; Meidt, Sharon; Kruijssen, Diederik; Schinnerer, Eva; Hygate, Alex; Kreckel, Kathryn; Rosolowsky, Erik;",12.843195541132294,0.4191561814263185,1.8,uid://A001/X1273/X64f,uid://A001/X1273/X652,uid://A002/Xcfe75c/X3a26,From the main sequence to the red cloud: linking the molecular cloud lifecycle to galaxy evolution,S,TARGET,49.741089163264505,22.714922921264197,Y,0,"Early-type galaxies, Giant Molecular Clouds (GMC) properties",Galaxy evolution,2017.1.00766.S,"[array([214.82, 216.8 ]), array([216.63, 218.62]), array([228.65, 229.65]), array([229.51, 231.49])]",NGC4596,NGC 4596,189.98311,10.17614,0.006311,True,False
2017.1.00766.S,NGC4596,189.98277820462008,10.175748874288601,293.29824337586575,72.83186501164042,[6],0.475449864471366,976.5974682263748,12m,mosaic,873.398,NaT,"[214.82..216.80GHz,31251.12kHz,1.1mJy/beam@10km/s,67.1uJy/beam@native, XX YY] U [216.69..218.56GHz,1953.19kHz,1.1mJy/beam@10km/s,69.3uJy/beam@native, XX YY] U [228.69..229.62GHz,976.60kHz,1.1mJy/beam@10km/s,99.9uJy/beam@native, XX YY] U [229.51..231.49GHz,31251.12kHz,1.1mJy/beam@10km/s,68.9uJy/beam@native, XX YY]",1275.081823526404,XX YY,2019-04-29T00:58:58,"Chevance, Melanie",NGC4596_a_06_TM1,"Schruba, Andreas; Jeffreson, Sarah; Leroy, Adam; Bigiel, Frank; Meidt, Sharon; Kruijssen, Diederik; Schinnerer, Eva; Hygate, Alex; Kreckel, Kathryn; Rosolowsky, Erik;",1.1060503652011848,0.03671750981191617,0.7358757,uid://A001/X1273/X64f,uid://A001/X1273/X650,uid://A002/Xdb7ab7/X8b2d,From the main sequence to the red cloud: linking the molecular cloud lifecycle to galaxy evolution,S,TARGET,53.32590752040346,2.887465187340699,,0,"Early-type galaxies, Giant Molecular Clouds (GMC) properties",Galaxy evolution,2017.1.00766.S,"[array([214.82, 216.8 ]), array([216.69, 218.56]), array([228.69, 229.62]), array([229.51, 231.49])]",NGC4596,NGC 4596,189.98311,10.17614,0.006311,True,False
2018.1.00484.S,NGC4596,189.98306249998967,10.176030555560978,293.29903606419043,72.83218593916669,[6],23.569910616065183,976.5232493062214,TP,,15977.472,2020-04-17,"[214.82..216.80GHz,31249.55kHz,10.3mJy/beam@10km/s,615.7uJy/beam@native, XX YY] U [216.63..218.62GHz,1953.10kHz,10.2mJy/beam@10km/s,615.7uJy/beam@native, XX YY] U [228.65..229.65GHz,976.55kHz,10.2mJy/beam@10km/s,889.8uJy/beam@native, XX YY] U [229.51..231.49GHz,31249.55